In [119]:
import pandas as pd

df = pd.read_csv('brand.csv')

In [121]:
import pandas as pd
import nltk
from nltk import bigrams
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
import re
def create_bigrams(text):
    # Lowercasing
    text = str(text).lower()
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-z\s]', '', text)
    # Tokenize
    words = nltk.word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lm = []
    for w in words:
        lm.append(lemmatizer.lemmatize(w))
    # Create bigrams
    return list(bigrams(lm))

# Apply the function to your comments
df['Bigram list'] = df['message'].map(create_bigrams)

In [122]:
df['Bigram list']

0       [(hi, pat), (pat, you), (you, forgot), (forgot...
1       [(im, sure), (sure, some), (some, folk), (folk...
2       [(you, can), (can, try), (try, to), (to, reviv...
3       [(model, v), (v, model), (model, is), (is, exa...
4       [(the, nissan), (nissan, is), (is, my), (my, f...
                              ...                        
4995    [(let, me), (me, try), (try, one), (one, more)...
4996    [(no, one), (one, like), (like, repair), (repa...
4997    [(toyota, gm), (gm, give), (give, you), (you, ...
4998    [(what, weakness), (weakness, you), (you, have...
4999    [(well, the), (the, cheapest), (cheapest, serv...
Name: Bigram list, Length: 5000, dtype: object

In [123]:
aspir_list = [('love', 'to'),
('would', 'love'),
('would', 'hope'),
('hope', 'to'),
('wish', 'to'),
('wish', 'that'),
('wish', 'for'),
('wishing', 'for'),
('want', 'to'),
('wants', 'to'),
('dream', 'of'),
('daydreaming', 'about'),
('dream', 'to'),
('desire', 'for'),
('desire', 'to'),
('everyone', 'desires'),
('future', 'car'),
('near', 'future'),
('future', 'vehicle'),
('to', 'buy'),
('buy', 'the'),
('buy', 'it'),
('can', 'buy'),
('would', 'buy'),
('will', 'buy'),
('will', 'buy'),
('could', 'buy'),
('buy', 'cars'),
('buy', 'new'),
('may', 'buy'),
('to', 'purchase'),
('purchase', 'the'),
('purchase', 'it'),
('purchase', 'cars'),
('possible', 'purchase'),
('will', 'purchase'),
('next', 'purchase'),
('like', 'to'),
('will', 'get'),
('could', 'get'),
('i', 'wish'),
('can', 'get'),
('i','want'),
('stick','with'),
             ("hope", "to"),
    ("dream", "of"),
    ("wish", "to"),
    ("looking", "forward"),
    ("saving", "for"),
    ("eager", "to"),
    ("excited", "about"),
    ("enthusiastic", "about"),
    ("keen", "on"),
    ("set", "on"),
    ("fixated", "on"),
    ("focused", "on"),
    ("determined", "to"),
    ("committed", "to"),
    ("dedicated", "to"),
    ("can't", "wait"),
    ("longing", "for"),
    ("yearning", "for"),
    ("itching", "for"),
    ("dying", "for"),
    ("dreaming", "of"),
    ("thinking", "of"),
    ("considering", "buying"),
    ("planning", "to"),
    ("aiming", "to"),
    ("hoping", "to"),
    ("wishing", "for"),
    ("desiring", "to"),
    ("craving", "for"),
    ("praying", "for"),
    ("envision", "driving"),
    ("fantasize", "about"),
    ("visualize", "driving"),
    ("imagine", "driving"),
    ("contemplating", "getting")]

In [124]:
def has_aspirational_bigram(bigram_list):
    if any(bigram in aspir_list for bigram in bigram_list):
        return 1
    return 0

# Apply the function to find comments with aspirational bigrams
df['is_aspirational'] = df['Bigram list'].apply(has_aspirational_bigram)

In [125]:
df['is_aspirational'].value_counts()
df.iloc[ :, -1:].columns.values

array(['is_aspirational'], dtype=object)

In [126]:
top10_brands=['subaru', 'chrysler', 'chevrolet', 'saturn', 'nissan', 'mazda', 'hyundai', 'toyota', 'ford', 'honda']
aspirations = df.iloc[ :, -1:].columns.values
aspr_matrix = pd.DataFrame(index=top10_brands, columns=aspirations)

In [128]:
from nltk.tokenize import word_tokenize
def tokenize_comment(text):
    text = str(text).lower()
    return word_tokenize(text)

# Apply tokenization to the comment column
df['message_tokens'] = df['message'].apply(tokenize_comment)
for brand in top10_brands:
    df[brand] = df['message_tokens'].map(lambda x: 1 if brand in x else 0)

In [132]:
df['is_aspirational'].value_counts()

0    4186
1     814
Name: is_aspirational, dtype: int64

In [133]:
for brand in top10_brands:
    freq_brand = df[brand].sum()
    freq_aspr = df['is_aspirational'].sum()
    freq_both = df["is_aspirational"][(df[brand] == 1) & (df['is_aspirational'] == 1 )].sum()
    lift = len(df) * freq_both / (freq_brand * freq_aspr)
    aspr_matrix.loc[brand] = lift
    

aspr_matrix.sort_values(by = 'is_aspirational', ascending= False)

,is_aspirational
chevrolet,1.406116
mazda,1.4046
subaru,1.390756
saturn,1.307441
toyota,1.268493
ford,1.191493
hyundai,1.175793
honda,1.145859
chrysler,1.037401
nissan,0.957413
